In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

ROOT_PROJECT = os.path.join(os.path.normpath(os.path.join(os.getcwd(), "../")))
sys.path[0] = ROOT_PROJECT

import numpy as np
# import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib_inline

matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (12.0, 7.0)


/tmp/ipykernel_71719/4091855078.py:20: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [15]:
EXH_LS = "Exhaustive LS"
HC = "HC + Grad. desc."
MAB = "MAB + Grad. desc."
GA = "Genetic algorithm"
SA = "Simulated annealing"

SSK = "GP (SSK)"
OK = "GP (Over.)"
TOK = "GP (Trans. Over.)"
DIFF = "GP (Diff.)"
HED = "GP (HED)"
LHS = "Lin. reg. (SH)"

OPT_TO_CITE = {
    (SSK, HC): ("BOiLS", "grosnit2021BOiLS"),
    (TOK, HC): ("Casmo.", "wan2021think"),
    (HED, HC): ("BODi", "Deshwal2023BODi"),
    (DIFF, EXH_LS): ("COMBO", "oh2019combinatorial"),
    (OK, MAB): ("CoCaBO", "pmlr-v119-ru20a"),
    (SSK, GA): ("BOSS", "DBLP:conf/nips/MossLBGR20"),
    (LHS, SA): ("BOCS", "baptista2018bayesian")
}

acq_opts = [EXH_LS, HC, SA, GA, MAB]
prob_models = [OK, TOK, HED, SSK, DIFF, LHS]

ACQ_OPT_BOX_T_WIDTH = 4.8
ACQ_OPT_BOX_MIN_HEIGHT = 3
ACQ_OPT_BOX = "\\tikzstyle{acq_opt_box}=[rectangle, " \
              + f"text width={ACQ_OPT_BOX_T_WIDTH}em, text centered, minimum height={ACQ_OPT_BOX_MIN_HEIGHT}em, rounded corners]\n"

MODEL_BOX_T_WIDTH = 6.
MODEL_BOX_MIN_HEIGHT = 1.4
MODEL_BOX = "\\tikzstyle{model_box}=[rectangle, " \
            + f"text width={MODEL_BOX_T_WIDTH}em, align=right, minimum height={MODEL_BOX_MIN_HEIGHT}em, rounded corners]\n"

OPTIMIZER_BOX_T_WIDTH = 3.
OPTIMIZER_BOX_MIN_HEIGHT = MODEL_BOX_MIN_HEIGHT
OPTIMIZER_BOX = "\\tikzstyle{optimizer_box}=[rectangle, text=blue," \
                + f"text width={OPTIMIZER_BOX_T_WIDTH}em, " \
                + f"text centered, minimum height={OPTIMIZER_BOX_MIN_HEIGHT}em, rounded corners, font=\\normalsize]\n"

NEW_OPTIMIZER_BOX_T_WIDTH = 1.5
NEW_OPTIMIZER_BOX_MIN_HEIGHT = MODEL_BOX_MIN_HEIGHT
NEW_OPTIMIZER_BOX = "\\tikzstyle{new_optimizer_box}=[rectangle, text=orange," \
                    + f"text width={NEW_OPTIMIZER_BOX_T_WIDTH}em, " \
                    + f"text centered, minimum height={NEW_OPTIMIZER_BOX_MIN_HEIGHT}em, font=\\large, font=\\bfseries]\n"

NONE_OPTIMIZER_BOX_T_WIDTH = 1.5
NONE_OPTIMIZER_BOX_MIN_HEIGHT = MODEL_BOX_MIN_HEIGHT
NONE_OPTIMIZER_BOX = "\\tikzstyle{none_optimizer_box}=[rectangle, text=gray," \
                     + f"text width={NONE_OPTIMIZER_BOX_T_WIDTH}em, " \
                     + f"text centered, minimum height={NONE_OPTIMIZER_BOX_MIN_HEIGHT}em, font=\\large, font=\\bfseries]\n"

NODE_TYPES = [ACQ_OPT_BOX, MODEL_BOX, OPTIMIZER_BOX, NEW_OPTIMIZER_BOX, NONE_OPTIMIZER_BOX]

s = "".join(NODE_TYPES) + "\n"


def acq_opt_box(label: str, x_start: float, y_start: float):
    return f"\\node[acq_opt_box] (state_box) at ({x_start}, {y_start})" + " {" + label + "};\n"


def model_box(label: str, x_start: float, y_start: float):
    return f"\\node[model_box] (state_box) at ({x_start}, {y_start})" + " {" + label + "};\n"


def optimizer_node(label, x_start: float, y_start: float, line_color: str, fill_color: str):
    return f"\\node[optimizer_box, thick, " + f"draw={line_color}, fill={fill_color}] (opt_box) at ({x_start}, {y_start})" + " {" + label + "};\n"


def new_optimizer_node(label, x_start: float, y_start: float, line_color: str, fill_color: str):
    return f"\\node[new_optimizer_box, " + f"draw={line_color}, fill={fill_color}] (opt_box)" \
        + f" at ({x_start}, {y_start})" + " {" + label + "};\n"


def none_optimizer_node(label, x_start: float, y_start: float, line_color: str, fill_color: str):
    return f"\\node[none_optimizer_box, " + f"draw={line_color}, fill={fill_color}] (opt_box)" \
        + f" at ({x_start}, {y_start})" + " {" + label + "};\n"


def get_upper_triangle(x: float, y: float, h: float):
    side = np.sqrt(3) / 2 * h
    return f"\\fill [opacity=1,black] ({x - side / 2}, {y}) -- ({x + side / 2}, {y}) -- ({x}, {y + h}) -- cycle;\n"


def get_right_triangle(x: float, y: float, h: float):
    side = np.sqrt(3) / 2 * h
    return f"\\fill [opacity=1,black] ({x}, {y - side / 2}) -- ({x}, {y + side / 2}) -- ({x + h}, {y}) -- cycle;\n"


TRIANGLE_H = .22

X_OFFSET = 1.2
Y_START = 0
x_offsets = np.linspace(0, 1, len(acq_opts)) * 7.2 + X_OFFSET

Y_OFFSET = 1.2
X_START = -1
y_offsets = np.linspace(0, 1, len(prob_models)) * 4.2 + Y_OFFSET

Y_DOT_LINES_START = Y_START + ACQ_OPT_BOX_T_WIDTH / 10
Y_DOT_LINES_END = y_offsets[-1] + .5

X_DOT_LINES_START = X_START + MODEL_BOX_T_WIDTH / 5
X_DOT_LINES_END = x_offsets[-1] + 1.4
X_DOT_LINES_START_WHITE = X_DOT_LINES_END - .5

LOWEST_X = -1.5
LOWEST_Y = -.3

ratio = .0

AUX_X = LOWEST_X + ratio * (X_DOT_LINES_START - LOWEST_X)
AUX_Y = LOWEST_Y + ratio * (Y_DOT_LINES_START - LOWEST_Y)

s += "\\draw " + f"({AUX_X}, {AUX_Y}) -- ({X_DOT_LINES_START}, {Y_DOT_LINES_START}) node[midway,sloped,above,font=\\bfseries] " \
     + "{Model} node[midway,sloped,below,font=\\bfseries] {Acq. opt.};\n"
s += "\\draw " + f"({LOWEST_X}, {LOWEST_Y}) -- ({AUX_X}, {AUX_Y});\n"

s += "\\draw " + f"({X_DOT_LINES_START}, {Y_DOT_LINES_START}) -- ({X_DOT_LINES_START_WHITE}, {Y_DOT_LINES_START});\n"
s += "\\draw " + f"({X_DOT_LINES_START}, {Y_DOT_LINES_START}) -- ({X_DOT_LINES_START}, {Y_DOT_LINES_END});\n"
s += "\\draw [white] " + f"({X_DOT_LINES_START_WHITE}, {Y_DOT_LINES_START}) -- ({X_DOT_LINES_END + .1}, {Y_DOT_LINES_START});\n"

for i in range(len(acq_opts)):
    s += acq_opt_box(label=acq_opts[i], x_start=x_offsets[i], y_start=Y_START)

for j in range(len(prob_models)):
    s += model_box(label=prob_models[j], x_start=X_START, y_start=y_offsets[j])
    s += get_right_triangle(x=X_DOT_LINES_START, y=y_offsets[j], h=TRIANGLE_H)

for i in range(len(acq_opts)):
    s += "\\draw[dotted] " + f"({x_offsets[i]}, {Y_DOT_LINES_START}) -- ({x_offsets[i]}, {Y_DOT_LINES_END - .5});\n"
    s += get_upper_triangle(x=x_offsets[i], y=Y_DOT_LINES_START, h=TRIANGLE_H)

for j in range(len(prob_models)):
    s += "\\draw[dotted] " + f"({X_DOT_LINES_START}, {y_offsets[j]}) -- ({X_DOT_LINES_END - 1}, {y_offsets[j]});\n"

for i in range(len(acq_opts)):
    acq_opt = acq_opts[i]
    for j in range(len(prob_models)):
        prob_model = prob_models[j]
        k = (prob_model, acq_opt)
        if k in OPT_TO_CITE:
            name, cite = OPT_TO_CITE[k]
            label = name + "~\\cite{" + cite + "}"
            line_color = "white"
            fill_color = "white"
            func = optimizer_node
        #         if k in OPT_TO_CITE:
        #             name, cite = OPT_TO_CITE[k]
        #             label = "\\citet{" + cite + "}"
        #             line_color = "white"
        #             fill_color = "white"
        #             func = optimizer_node

        elif acq_opt == MAB and prob_model in [SSK, DIFF, LHS]:
            label = "x"
            line_color = "white"
            fill_color = "white"
            func = none_optimizer_node
        else:
            label = "New"
            line_color = "white"
            fill_color = "white"
            func = new_optimizer_node

        s += func(label, x_start=x_offsets[i], y_start=y_offsets[j], line_color=line_color, fill_color=fill_color)

print(s)

\tikzstyle{acq_opt_box}=[rectangle, text width=4.8em, text centered, minimum height=3em, rounded corners]
\tikzstyle{model_box}=[rectangle, text width=6.0em, align=right, minimum height=1.4em, rounded corners]
\tikzstyle{optimizer_box}=[rectangle, text=blue,text width=3.0em, text centered, minimum height=1.4em, rounded corners, font=\normalsize]
\tikzstyle{new_optimizer_box}=[rectangle, text=orange,text width=1.5em, text centered, minimum height=1.4em, font=\large, font=\bfseries]
\tikzstyle{none_optimizer_box}=[rectangle, text=gray,text width=1.5em, text centered, minimum height=1.4em, font=\large, font=\bfseries]

\draw (-1.5, -0.3) -- (0.19999999999999996, 0.48) node[midway,sloped,above,font=\bfseries] {Model} node[midway,sloped,below,font=\bfseries] {Acq. opt.};
\draw (-1.5, -0.3) -- (-1.5, -0.3);
\draw (0.19999999999999996, 0.48) -- (9.3, 0.48);
\draw (0.19999999999999996, 0.48) -- (0.19999999999999996, 5.9);
\draw [white] (9.3, 0.48) -- (9.9, 0.48);
\node[acq_opt_box] (state_box)